# **Lecture des données**

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation des données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
# SeaBorn : librairie de graphiques avancés
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# Lecture des données d'apprentissage et de test
df = pd.read_csv("../input/housesalesprediction/kc_house_data.csv")

In [ ]:
df.head().T

# **Interprétation des paramètres**
- id : id de la vente
- date : date de la vente
- price : prix de la vente
- bedrooms : nombre de chambre
- bathrooms : nombre de salle de bain
- sqft_living : surface de la maison
- sqft_lot : surface totale du terrain
- floors : nombre d'étage
- waterfront : vue sur la mer
- view :
- condition : note en fonction de l'age et de la qualité de la construction (1-5)
- grade : note en fonction de la qualité de la construction (1-13)
- sqft_above : surface de la maison - surface du sous-sol
- sqft_basement : surface du sous-sol
- yr_built : année de construction
- yr_renovated : année de rénovation
- zipcode : code postal
- lat : latitude
- long : longitude
- sqft_living15 : surface moyenne des 15 maisons les plus proches
- sqft_lot15 : surface moyenne des 15 terrains (maisons y compris) les plus proches

In [ ]:
df.count()    # Comptage des valeurs par colonnes

> On remarque qu'il ne manque aucune valeur pour chaque colonne de la table

# **Visualisations des données de la table** 

> Quelques réprésentation du dataset

In [ ]:
plt.figure(figsize = (8, 5))
sns.jointplot(df.sqft_living, df.price, 
              alpha = 0.5)
plt.xlabel('Sqft Living')
plt.ylabel('Sale Price')
plt.show()

In [ ]:
condition = df['condition'].value_counts()

print("Nombre de logement par condition : ")
print(condition)

fig, ax = plt.subplots(ncols=2, figsize=(14,5))
sns.countplot(x='condition', data=df, ax=ax[0])
sns.boxplot(x='condition', y= 'price',
            data=df, ax=ax[1])
plt.show()

> *Trace les courbes de distribution de l'année de construction selon condition (utilisation de FacetGrid)*

In [ ]:
fig = sns.FacetGrid(df, hue="condition", aspect=3, palette="Set1")
fig.map(sns.kdeplot, "yr_built", shade=True)
fig.add_legend()

> *Trace les courbes de distribution de l'année de construction selon le grade (utilisation de FacetGrid)*

In [ ]:
fig = sns.FacetGrid(df, hue="grade", aspect=3, palette="Set1")
fig.map(sns.kdeplot, "yr_built", shade=True)
fig.add_legend()

# **Conditionnement des données**
Elimination des colonnes non pertinentes pour la prédiction, et placer le resultat dans la varible HouseSales

In [ ]:
df.columns

In [ ]:
# On élimine les colonnes non pertinentes pour la prédiction
HouseSales = df.drop(['id','date','lat','long'], axis=1)

In [ ]:
HouseSales.count()

> On remarque qu'il ne manque aucune valeur

In [ ]:
print("Prix Min :", HouseSales['price'].min())
print("Prix Moyen :", HouseSales['price'].mean())
print("Prix Médiant :",HouseSales['price'].median())
print("Prix Max :", HouseSales['price'].max())

# **Déséquilibre des distributions**
Certaines distributions sont déséquilibrées, et éloignées d'une loi normale :

In [ ]:
sns.distplot(HouseSales['price'], color='blue')

In [ ]:
HouseSales['log_price'] = np.log(HouseSales['price']+1)
sns.distplot(HouseSales['log_price'], color="blue")

In [ ]:
# HouseSales = HouseSales.drop(['price'], axis=1)

**Mise à l'échelle des données quantitatives**

In [ ]:
HouseSales[['yr_built', 'log_price']].describe()

In [ ]:
sns.distplot(HouseSales.log_price, color='blue')
sns.distplot(HouseSales.yr_built, color='red')

In [ ]:
# import de la librairie sklearn qui est une librairie de prétraitement des données
from sklearn import preprocessing

In [ ]:
# normalisation des données pour ramener les valeurs min et max à 0 et 1
minmax = preprocessing.MinMaxScaler(feature_range=(0,1))
HouseSales[['yr_built', 'log_price']] = minmax.fit_transform(HouseSales[['yr_built', 'log_price']])

In [ ]:
sns.distplot(HouseSales.log_price, color='blue')
sns.distplot(HouseSales.yr_built, color='red')

In [ ]:
# normalisation des données pour ramener les valeurs ecart type et moyenne à 0 et 1
scaler = preprocessing.StandardScaler()
HouseSales[['yr_built', 'log_price']] = scaler.fit_transform(HouseSales[['yr_built', 'log_price']])

In [ ]:
sns.distplot(HouseSales.log_price, color='blue')
sns.distplot(HouseSales.yr_built, color='red')

In [ ]:
HouseSales.info()

# **Création des jeux d'apprentissage et de test**

In [ ]:
X = HouseSales.drop(['price'], axis = 1)
Y = HouseSales['price']

In [ ]:
# Séparation du dataset en deux parties :
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [ ]:
print(X_train.shape)
print(Y_test.shape)

# **Régression logistique**
Appliquation d'une régression logistique pour classifier l'ensemble de test

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,Y_train)

In [ ]:
Y_lr = lr.predict(X_test)

# **Mesures de performance**

In [ ]:
# Importation des méthodes de mesure de performances
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

In [ ]:
# Affichage de la matrice de confusion :
print(confusion_matrix(Y_test, Y_lr))

In [ ]:
# Affichage du score de performance
print(accuracy_score(Y_test, Y_lr))

In [ ]:
# Affichage du rapport de classification
print(classification_report(Y_test, Y_lr))